#### The term hate speech is understood as any type of verbal, written or behavioural communication that attacks or uses derogatory or discriminatory language against a person or group based on what they are, in other words, based on their religion, ethnicity, nationality, race, colour, ancestry, sex or another identity factor. In this article, I will take you through a hate speech detection model with Machine Learning and Python.

#### Hate Speech Detection is generally a task of sentiment classification. So for training, a model that can classify hate speech from a certain piece of text can be achieved by training it on a data that is generally used to classify sentiments. So for the task of hate speech detection model, I will use the Twitter data.


#### Nefret söylemi terimi, bir kişi veya grubu ne olduğuna, başka bir deyişle dinine, etnik kökenine, dinine, etnik kökenine, karşı saldırıya geçen veya aşağılayıcı veya ayrımcı bir dil kullanan her türlü sözlü, yazılı veya davranışsal iletişim olarak anlaşılmaktadır. milliyet, ırk, renk, soy, cinsiyet veya başka bir kimlik faktörü. Bu yazımda sizlere Machine Learning ve Python ile bir nefret söylemi tespit modelinden bahsedeceğim.

#### Nefret Söylemi Tespiti genellikle bir duygu sınıflandırması görevidir. Dolayısıyla, eğitim için, belirli bir metin parçasından nefret söylemini sınıflandırabilen bir model, onu genellikle duyguları sınıflandırmak için kullanılan bir veri üzerinde eğiterek elde edilebilir. Dolayısıyla nefret söylemi tespit modeli görevi için Twitter verilerini kullanacağım.

# Hate Speech Detection Model
#### The data set I will use for the hate speech detection model consists of a test and train set. The training package includes a list of 31,962 tweets, a corresponding ID and a tag 0 or 1 for each tweet. The particular sentiment we need to detect in this dataset is whether or not the tweet is based on hate speech.

# Nefret Söylemi Tespit Modeli
#### Nefret söylemi tespit modeli için kullanacağım veri seti bir test ve tren setinden oluşuyor. Eğitim paketi, 31.962 tweet'lik bir liste, karşılık gelen bir kimlik ve her tweet için 0 veya 1 etiketi içerir. Bu veri setinde tespit etmemiz gereken özel duygu, tweet'in nefret söylemine dayalı olup olmadığıdır.

In [1]:
import pandas as pd

In [2]:
train=pd.read_csv("train.csv")

In [3]:
print("Training Set:",train.columns,train.shape,len(train))

Training Set: Index(['id', 'label', 'tweet'], dtype='object') (31962, 3) 31962


In [4]:
test=pd.read_csv("test.csv")

In [5]:
print("Test Set:",test.columns,test.shape,len(test))

Test Set: Index(['id', 'tweet'], dtype='object') (17197, 2) 17197


# Data Cleaning
#### Data cleaning is the process of preparing incorrectly formated data for analysis by deleting or modifying the incorrectly formatted data which is generally not necessary or useful for data analysis, as it can hinder the process or provide inaccurate results. Now I will perform the process of data cleaning by using the re library in Python:


# Veri temizleme
#### Veri temizleme, süreci engelleyebileceği veya hatalı sonuçlar verebileceği için genellikle veri analizi için gerekli veya yararlı olmayan yanlış biçimlendirilmiş verileri silerek veya değiştirerek yanlış biçimlendirilmiş verileri analiz için hazırlama işlemidir. Şimdi Python'da re kütüphanesini kullanarak veri temizleme işlemini gerçekleştireceğim:

In [6]:
import re

In [8]:
def clean_text(df,text_field):
    df[text_field] =df[text_field].str.lower()
    df[text_field]=df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
    return df

In [9]:
test_clean=clean_text(test,"tweet")

In [10]:
train_clean=clean_text(train,"tweet")

# Handling Imbalanced data for Hate Speech Detection Model
#### If you will deeply analyse the task we are working on with context to the data we are using, you will find that the tweets regarding hate speeches are comparatively lesser than others, so this is a situation of an unbalanced data.

#### If we will fit this data to train our hate speech detection model, then the model will not generalize any hate speech because the data with context to the hate speech is very less than the positive ones. So in this situation, we need to prepare the data to fit properly in our model.

#### There are a number of methods you can use to deal with this. One approach is to use either oversampling or downsampling. In the case of oversampling, we use a function that repeatedly samples, with replacement, from the minority class until the class is the same size as the majority. Let’s see how we can handle this:



# Nefret Söylemi Tespit Modeli için Dengesiz Verileri İşleme
#### Üzerinde çalıştığımız görevi, kullandığımız veriler bağlamında derinlemesine analiz edecek olursanız, nefret söylemleriyle ilgili tweet'lerin diğerlerine göre nispeten daha az olduğunu göreceksiniz, yani bu dengesiz bir veri durumu.

#### Nefret söylemi tespit modelimizi eğitmek için bu verileri sığdırırsak, model herhangi bir nefret söylemini genelleştirmeyecektir çünkü nefret söylemi bağlamına sahip veriler olumlu olanlardan çok daha azdır. Dolayısıyla bu durumda verileri modelimize tam olarak uyacak şekilde hazırlamamız gerekiyor.

#### Bununla başa çıkmak için kullanabileceğiniz birkaç yöntem var. Bir yaklaşım, aşırı örnekleme veya alt örnekleme kullanmaktır. Aşırı örnekleme durumunda, sınıf çoğunluk ile aynı boyuta gelene kadar azınlık sınıfından değiştirme ile tekrar tekrar örnekleyen bir işlev kullanırız. Bakalım bununla nasıl başa çıkabiliriz:

In [11]:
from sklearn.utils import resample 

In [12]:
train_majority=train_clean[train_clean.label==0]

In [13]:
train_minority=train_clean[train_clean.label==1]

In [16]:
train_minority_upsampled=resample(train_minority,
                                 replace=True,
                                 n_samples=len(train_majority),
                                 random_state=123)

In [17]:
train_upsampled=pd.concat([train_minority_upsampled,train_majority])

In [19]:
train_upsampled["label"].value_counts()

1    29720
0    29720
Name: label, dtype: int64

# Creating a Pipeline
#### For simplicity and reproducibility of the hate speech detection model, I will use the Scikit-Learn’s pipeline with an SGDClassifier, before training our model:

# Boru Hattı Oluşturma
#### Nefret söylemi algılama modelinin basitliği ve tekrarlanabilirliği için, modelimizi eğitmeden önce Scikit-Learn'in ardışık düzenini bir SGDClassifier ile kullanacağım:

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier

In [21]:
pipeline_sgd=Pipeline([
    ("vect",CountVectorizer()),
    ("tfidf",TfidfTransformer()),
    ("nb",SGDClassifier()),
])

# Training the Hate Speech Detection Model
#### Now, before training the model, let’s split the data into a training set and a test set:

# Nefret Söylemi Tespit Modelinin Eğitimi
#### Şimdi, modeli eğitmeden önce verileri bir eğitim seti ve bir test seti olarak ayıralım:

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train,X_test,y_train,y_test=train_test_split(train_upsampled["tweet"],
                                              train_upsampled["label"],random_state=0)

#### Now let’s train the model and predict the results on the test set using the F1 score method:

#### Şimdi modeli eğitelim ve F1 skor yöntemini kullanarak test setindeki sonuçları tahmin edelim:

In [24]:
model=pipeline_sgd.fit(X_train,y_train)

In [25]:
y_predict=model.predict(X_test)

In [26]:
from sklearn.metrics import f1_score

In [27]:
f1_score(y_test,y_predict)

0.9693292438991865

#### So we got an F1 score of 0.96 per cent which is generally appreciatable. This model can now be deployed and used in production

#### Yani yüzde 0.96'lık bir F1 puanı aldık ve bu genellikle takdire şayan. Bu model artık devreye alınabilir ve üretimde kullanılabilir